# Notebook 3 — Product of Exponentials Forward Kinematics (PoE FK)


**Aim:** Compute end-effector pose from joint angles using the PoE formula.

**You will be able to:**
- Define screw axes \(S_i\) in the space frame
- Compute \(T(	heta)=e^{[S_1]	heta_1}\cdots e^{[S_n]	heta_n}M\)
- Visualize a simple arm motion and verify correctness


In [ ]:
# Common setup for all notebooks
import numpy as np
import math
import matplotlib.pyplot as plt

# For 3D plots
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

np.set_printoptions(precision=4, suppress=True)


In [ ]:
# --- Minimal "Modern Robotics style" utilities (self-contained) ---

def skew(w):
    'Return [w]x (3x3 skew-symmetric) for w = (wx,wy,wz).'
    wx, wy, wz = w
    return np.array([[0, -wz, wy],
                     [wz, 0, -wx],
                     [-wy, wx, 0]], dtype=float)

def so3_exp(omega, theta):
    'Rodrigues: exp([omega] theta). omega can be non-unit; theta in radians.'
    omega = np.asarray(omega, dtype=float).reshape(3)
    norm = np.linalg.norm(omega)
    if norm < 1e-12:
        return np.eye(3)
    w = omega / norm
    th = theta * norm
    W = skew(w)
    return np.eye(3) + math.sin(th)*W + (1-math.cos(th))*(W@W)

def so3_log(R):
    'Return (omega_hat, theta) such that R = exp([omega_hat] theta), omega_hat unit.'
    R = np.asarray(R, dtype=float).reshape(3,3)
    tr = np.trace(R)
    cos_th = (tr - 1)/2
    cos_th = np.clip(cos_th, -1.0, 1.0)
    theta = math.acos(cos_th)
    if abs(theta) < 1e-12:
        return np.array([1.0, 0.0, 0.0]), 0.0
    if abs(theta - math.pi) < 1e-6:
        # Near pi: robust axis extraction from diagonal
        v = np.sqrt(np.maximum((np.diag(R)+1)/2, 0))
        # Fix signs using off-diagonals
        if R[2,1] - R[1,2] < 0: v[0] = -v[0]
        if R[0,2] - R[2,0] < 0: v[1] = -v[1]
        if R[1,0] - R[0,1] < 0: v[2] = -v[2]
        omega_hat = v / (np.linalg.norm(v) + 1e-12)
        return omega_hat, theta
    W = (R - R.T) / (2*math.sin(theta))
    omega_hat = np.array([W[2,1], W[0,2], W[1,0]])
    omega_hat /= (np.linalg.norm(omega_hat)+1e-12)
    return omega_hat, theta

def make_T(R, p):
    'Homogeneous transform from rotation R (3x3) and position p (3,).'
    T = np.eye(4)
    T[:3,:3] = np.asarray(R, dtype=float).reshape(3,3)
    T[:3,3]  = np.asarray(p, dtype=float).reshape(3)
    return T

def inv_T(T):
    'Inverse of SE(3) transform.'
    T = np.asarray(T, dtype=float).reshape(4,4)
    R = T[:3,:3]
    p = T[:3,3]
    Rt = R.T
    return make_T(Rt, -Rt@p)

def adjoint(T):
    'Adjoint matrix Ad_T (6x6) mapping twists between frames.'
    T = np.asarray(T, dtype=float).reshape(4,4)
    R = T[:3,:3]
    p = T[:3,3]
    P = skew(p)
    Ad = np.zeros((6,6))
    Ad[:3,:3] = R
    Ad[3:,3:] = R
    Ad[3:,:3] = P@R
    return Ad

def se3_exp(S, theta):
    'SE(3) exponential: exp([S] theta), where S is 6-vector screw axis.'
    S = np.asarray(S, dtype=float).reshape(6)
    w = S[:3]
    v = S[3:]
    normw = np.linalg.norm(w)
    if normw < 1e-12:
        R = np.eye(3)
        p = v * theta
        return make_T(R, p)
    w_hat = w / normw
    th = theta * normw
    R = so3_exp(w_hat, th)
    W = skew(w_hat)
    G = (np.eye(3)*th + (1-math.cos(th))*W + (th - math.sin(th))*(W@W))
    p = (G @ (v / normw))
    return make_T(R, p)

def se3_log(T):
    'SE(3) log: return V (6,) and theta such that T = exp([V] theta).'
    T = np.asarray(T, dtype=float).reshape(4,4)
    R = T[:3,:3]
    p = T[:3,3]
    w_hat, th = so3_log(R)
    if abs(th) < 1e-12:
        V = np.concatenate([np.zeros(3), p])
        return V, 1.0
    W = skew(w_hat)
    A = (np.eye(3) - 0.5*W + (1/th - 0.5/math.tan(th/2))*(W@W))
    v = A @ p
    V = np.concatenate([w_hat, v])
    return V, th

def FK_space(Slist, thetalist, M):
    'PoE forward kinematics in space form.'
    T = np.eye(4)
    for S, th in zip(Slist.T, thetalist):
        T = T @ se3_exp(S, th)
    return T @ M

def Jacobian_space(Slist, thetalist):
    'Space Jacobian J_s(theta). Slist: 6xn. thetalist: n.'
    n = Slist.shape[1]
    J = np.zeros((6,n))
    T = np.eye(4)
    for i in range(n):
        if i == 0:
            J[:,0] = Slist[:,0]
        else:
            T = T @ se3_exp(Slist[:,i-1], thetalist[i-1])
            J[:,i] = adjoint(T) @ Slist[:,i]
    return J

def damped_pinv(J, lam=1e-3):
    'Damped least squares pseudo-inverse.'
    m, n = J.shape
    return J.T @ np.linalg.inv(J@J.T + (lam**2)*np.eye(m))

# --- plotting helpers ---
def plot_frame(ax, T=np.eye(4), name=None, length=0.08):
    'Plot a coordinate frame (3 axes).'
    T = np.asarray(T, float)
    o = T[:3,3]
    R = T[:3,:3]
    xs = o + R[:,0]*length
    ys = o + R[:,1]*length
    zs = o + R[:,2]*length
    ax.plot([o[0], xs[0]],[o[1], xs[1]],[o[2], xs[2]], linewidth=2)
    ax.plot([o[0], ys[0]],[o[1], ys[1]],[o[2], ys[2]], linewidth=2)
    ax.plot([o[0], zs[0]],[o[1], zs[1]],[o[2], zs[2]], linewidth=2)
    if name:
        ax.text(o[0], o[1], o[2], f' {name}')

def set_axes_equal(ax, lim=0.4):
    ax.set_xlim(-lim, lim)
    ax.set_ylim(-lim, lim)
    ax.set_zlim(0, 2*lim)
    ax.set_xlabel('x'); ax.set_ylabel('y'); ax.set_zlabel('z')


## 1) Define a simple 3-DoF spatial arm (toy but real SE(3))

Revolute joints:
- J1: about z at origin
- J2: about y at x=L1
- J3: about y at x=L1+L2


In [ ]:
L1, L2, L3 = 0.20, 0.20, 0.15

def screw_revolute(w, q):
    w = np.asarray(w, float)
    q = np.asarray(q, float)
    v = -np.cross(w, q)
    return np.concatenate([w, v])

S1 = screw_revolute([0,0,1], [0,0,0])
S2 = screw_revolute([0,1,0], [L1,0,0])
S3 = screw_revolute([0,1,0], [L1+L2,0,0])

Slist = np.stack([S1,S2,S3], axis=1)
M = make_T(np.eye(3), [L1+L2+L3, 0, 0])

print("Slist=\n", Slist)
print("M=\n", M)


## 2) FK + visualization


In [ ]:
def fk_all_frames(thetas):
    T = np.eye(4)
    Ts = [T.copy()]  # base
    for i in range(3):
        T = T @ se3_exp(Slist[:,i], thetas[i])
        Ts.append(T.copy())
    Ts.append(FK_space(Slist, thetas, M))  # ee
    return Ts

def plot_arm(thetas, title=""):
    Ts = fk_all_frames(thetas)
    pts = np.array([T[:3,3] for T in Ts])
    fig = plt.figure(figsize=(7,6))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot(pts[:,0], pts[:,1], pts[:,2], marker='o')
    plot_frame(ax, np.eye(4), "base", length=0.07)
    plot_frame(ax, Ts[-1], "ee", length=0.07)
    set_axes_equal(ax, lim=0.6)
    plt.title(title)
    plt.show()

plot_arm([0.0, 0.0, 0.0], title="Home pose")
plot_arm([math.radians(30), math.radians(40), math.radians(-30)], title="A sample configuration")


## 3) Small motion sweep


In [ ]:
configs = [
    [0,0,0],
    [math.radians(20), math.radians(10), 0],
    [math.radians(40), math.radians(20), math.radians(-20)],
    [math.radians(60), math.radians(30), math.radians(-40)],
]
for i,cfg in enumerate(configs):
    plot_arm(cfg, title=f"Config {i}: {np.round(cfg,3)} rad")
